## Get data from Meetup.com

We are going to get data from Meetup's API. You can create custom requests using their API console, then make them via Python by copying the secure link here.

https://secure.meetup.com/meetup_api/console/?path=/2/members

In [1]:
import pandas as pd
import numpy as np
import requests
import os

os.getcwd()

#### Get PyNash Members

In [5]:
members = pd.DataFrame()

for page in np.arange(10):
    q = 'https://api.meetup.com/2/members?offset={}&format=json&group_urlname=PyNash&page=200&key=1eb16676d664fa48314391ae5b6c'.format(page)
    response = requests.get(q).json()
    if len(response['results']) > 0:
        tdf = pd.DataFrame.from_dict(response['results']).set_index('id')
        members = members.append(tdf)
    else: 
        continue
    
members = members[['name', 'city',  'state', 'hometown', 'joined',  'visited', 'lat', 'lon']] # 'topics' might also be of interest

# Write to computer
write_data = True
if write_data == True:
    members.to_csv('data_members.csv', encoding='utf-8') 
    
print('There are {} members in the PyNash MeetUp group.'.format(members.shape[0]))

There are 1433 members in the PyNash MeetUp group.


#### Get memberships for each member

In [27]:
from time import sleep
import json


edges = pd.DataFrame()
err_ids = []
for pid in members.index:
    r = requests.get('https://api.meetup.com/2/groups?&sign=true&member_id={}&page=200&key=1eb16676d664fa48314391ae5b6c'.format(pid))
    try:
        r = r.json()
        for membership in r['results']:
            edge = pd.Series({'member_id': pid, 
                              'group_id': membership['id'], 
                              'group_name': membership['name']})
            edges = edges.append(edge, ignore_index=True)
    except json.decoder.JSONDecodeError:
        print(pid)
        err_ids.append(pid)
    
    # Sleep briefly so that API doesn't get overwhelmed
    sleep(0.2)
        

# Write to computer
write_data = True
if write_data == True:
    edges.to_csv('data_edges.csv') 
